## Data Cleaning and Pandas

See Chapter 7 of "Python for Data Analysis" by Wes McKinney

In [1]:
import pandas as pd
import numpy as np

### Detecting and dealing with missing data

python uses both null and NaN 

in Pandas,  the convention is to use NaN to indicate missing data

### Comment on dropping Na values

There are times when it seems to okay to simply drop Nas,   if a variable has over 50% of the variables missing,  it may be simply useless

But, in general,  we really want to know why data is missing before we remove it.

If there are patterns or correlations in the missing data, that means something.

Some ML tools can use Na values in the model,  such as GBMLight,   so they attempt to include the Na values as data input

In [6]:
stuff=['cat','dog','hamster',np.nan, 'aardvark']
stuff2=[1,2,None,3,4]

first_df=pd.DataFrame(list(zip(stuff,stuff2)),columns=['animal','flag'])
                                                       
first_df.head()                                                       

,animal,flag
0,cat,1.0
1,dog,2.0
2,hamster,NaN
3,NaN,3.0
4,aardvark,4.0


In [9]:
first_df.isnull()

,animal,flag
0,False,False
1,False,False
2,False,True
3,True,False
4,False,False


In [ ]:
#Question-  what is happeing here with NaN and null?  How do the detection functions isnull and isna seem to work

In [10]:
first_df.animal.isnull()

0    False
1    False
2    False
3     True
4    False
Name: animal, dtype: bool

In [11]:
first_df.flag.isna()

0    False
1    False
2     True
3    False
4    False
Name: flag, dtype: bool

In [12]:
first_df.animal.notnull()

0     True
1     True
2     True
3    False
4     True
Name: animal, dtype: bool

In [ ]:
# Automatic removal of rows or columns with NA

In [14]:
from numpy import nan as NA

# note the aliasing of nan as NA,   this may be easy to read

data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],[NA, NA, NA], [NA, 6.5, 3.]])
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [15]:
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [16]:
data[4]=NA
data
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


# look up pandas dropna,    what other options are there for how=?,    what is axis=1 doing?   What is the default axis?

In [17]:
df = pd.DataFrame(np.random.randn(7, 3))

df.iloc[:4, 1] = NA

df.iloc[:2, 2] = NA

df

,0,1,2
0,1.411329,NaN,NaN
1,-0.088368,NaN,NaN
2,1.919117,NaN,1.947893
3,1.598605,NaN,1.150123
4,-1.513905,0.209443,0.437012
5,-1.602234,-1.279344,0.358195
6,-0.939874,-0.471582,0.869062


In [18]:
df.dropna(thresh=2)

,0,1,2
2,1.919117,NaN,1.947893
3,1.598605,NaN,1.150123
4,-1.513905,0.209443,0.437012
5,-1.602234,-1.279344,0.358195
6,-0.939874,-0.471582,0.869062


# Filling in missing data

imputing or filling in constants

In [21]:
df.fillna(0)


,0,1,2
0,1.411329,0.000000,0.000000
1,-0.088368,0.000000,0.000000
2,1.919117,0.000000,1.947893
3,1.598605,0.000000,1.150123
4,-1.513905,0.209443,0.437012
5,-1.602234,-1.279344,0.358195
6,-0.939874,-0.471582,0.869062


In [24]:
df.fillna(data.mean())

,0,1,2
0,1.411329,6.500000,3.000000
1,-0.088368,6.500000,3.000000
2,1.919117,6.500000,1.947893
3,1.598605,6.500000,1.150123
4,-1.513905,0.209443,0.437012
5,-1.602234,-1.279344,0.358195
6,-0.939874,-0.471582,0.869062


In [26]:
df.fillna({1:0.5,2.0:0})

,0,1,2
0,1.411329,0.500000,0.000000
1,-0.088368,0.500000,0.000000
2,1.919117,0.500000,1.947893
3,1.598605,0.500000,1.150123
4,-1.513905,0.209443,0.437012
5,-1.602234,-1.279344,0.358195
6,-0.939874,-0.471582,0.869062


In [28]:
df.fillna({1:data[1].mean(),2.0:data[2].mean()})

,0,1,2
0,1.411329,6.500000,3.000000
1,-0.088368,6.500000,3.000000
2,1.919117,6.500000,1.947893
3,1.598605,6.500000,1.150123
4,-1.513905,0.209443,0.437012
5,-1.602234,-1.279344,0.358195
6,-0.939874,-0.471582,0.869062


# # Question
Explain what the previous two cells are doing
Create a cell below that replaces Na with the median


## dropping duplicate rows


In [29]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'], 'k2': [1, 1, 2, 3, 3, 4, 4]})

In [30]:
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

In [31]:
np.sum(data.duplicated())


1

In [32]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


## Replacing values and Sentinels

In many older systems, there are "sentinel" or "marker" values that indicate missing data,   common values used for this are 9999, 99999, -9999, "?", etc.  

In pf.read_csv(),  and probably many other read() operations, we can specify the values interpretted as Na

na_values:  scalar, str, list-like, or dict, optional

Additional strings to recognize as NA/NaN. If dict passed, specific per-column NA values. By default the following values are interpreted as NaN: ‘’, ‘#N/A’, ‘#N/A N/A’, ‘#NA’, ‘-1.#IND’, ‘-1.#QNAN’, ‘-NaN’, ‘-nan’, ‘1.#IND’, ‘1.#QNAN’, ‘<NA>’, ‘N/A’, ‘NA’, ‘NULL’, ‘NaN’, ‘n/a’, ‘nan’, ‘null’.


In [33]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

In [34]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [35]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

# Altering Axis indexes



In [36]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),index=['Ohio', 'Colorado', 'New York'],columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [39]:
#Applying a transformation to indices

def trans(x):
    return( x[0:4].upper() )

data.index=data.index.map(trans)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [43]:
data.index

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [45]:
 # This is a function I've been trying to find for a while, handy
# Notice that extra space in the index 'NEW '    
     
data.rename(index={'NEW ': 'NY'},columns={'three': 'peekaboo'})

,one,two,peekaboo,four
OHIO,0,1,2,3
COLO,4,5,6,7
NY,8,9,10,11


In [46]:
data.rename(index={'NEW ': 'NY'})

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NY,8,9,10,11


In [ ]:
## Discretization and Binning of continuous data

In [52]:


ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

#convert to categorical ranges

bins = [18, 25, 35, 60, 100]
    
cats = pd.cut(ages, bins)   

cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [53]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [54]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [55]:
ages

[20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [56]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

# when you create a categorizer,  where are the categories names? Where are the values for each specimen?

In [57]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

# is this set up to indicate an ordered category?

## Dividing data into equally spaced quintiles

qcut produces a desired number of quintile categories

In [58]:
data = np.random.randn(1000)  # Normally distributed

cats = pd.qcut(data, 4)  # Cut into quartiles

cats

[(-3.637, -0.677], (0.0134, 0.656], (0.0134, 0.656], (0.656, 3.258], (-3.637, -0.677], ..., (0.656, 3.258], (-3.637, -0.677], (0.0134, 0.656], (0.656, 3.258], (0.0134, 0.656]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.637, -0.677] < (-0.677, 0.0134] < (0.0134, 0.656] < (0.656, 3.258]]

In [59]:
pd.value_counts(cats)

(-3.637, -0.677]    250
(-0.677, 0.0134]    250
(0.0134, 0.656]     250
(0.656, 3.258]      250
dtype: int64

## Detecting and Filtering Outliers


In [61]:
data = pd.DataFrame(np.random.randn(1000, 4))

data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.060900,-0.034858,0.022065,-0.010982
std,1.010091,1.029371,0.988261,1.017005
min,-3.213225,-3.953834,-2.845964,-4.052107
25%,-0.637361,-0.743595,-0.605601,-0.697344
50%,0.104128,-0.032620,0.040356,0.017852
75%,0.781098,0.643349,0.680637,0.659851
max,2.857865,3.322206,3.974847,4.114486


In [62]:
# Detecting values with abs value over 3 from column 2

col = data[2]

col[np.abs(col) > 3]

747    3.974847
Name: 2, dtype: float64

In [63]:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
95,-0.783070,3.278624,1.381623,-0.481441
460,-0.914664,-3.953834,0.088641,-1.800631
514,-1.563344,-3.025871,-0.369041,0.565531
622,-0.408550,3.322206,0.019303,-0.813832
738,1.702989,0.612655,0.182119,4.114486
747,1.687800,-0.276133,3.974847,-0.957911
833,-3.213225,-1.087208,-0.929384,-1.166819
850,1.079853,0.320367,-0.851948,-4.052107
963,-3.170967,-0.537743,0.573688,0.090742
